In [15]:
# Import packages
import datetime
import geopandas as gpd
import holidays
# import july
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
from dbfread import DBF
# import seaborn as sns
import shutil
import time
# Import CEA modules
import cea.config
import cea.api 
from cea.utilities.dbf import *
from cea.utilities.epwreader import epw_reader
from cea.datamanagement.archetypes_mapper import archetypes_mapper
from cea.demand.schedule_maker.schedule_maker import main as schedule_maker
from cea.demand.demand_main import main as demand_main
from cea.resources.radiation.main import main as radiation_main


DAYSIM_BIN_DIRECTORY = os.path.join(os.getcwd(), 'CEA_model/0_daysim_binaries/darwin')
project_folder = os.path.join(os.getcwd(), 'CEA_model')



In [14]:
zone_gdf = gpd.read_file(
    os.path.join(project_folder, 'Block0202023', '2020', 'inputs', 'building-geometry', 'zone.shp')) 
list_buildings = zone_gdf["Name"].tolist() 
import cea.config
config = cea.config.Configuration()
config.multiprocessing = True

In [12]:
archetypes = ["AB_E" , "AB_E_NR", "AB_E_AR"] 
years = ["2020", "2021", "2022"]

In [16]:

for archetype in archetypes:
    for year in years: 
        path_to_new_scenario = os.path.join(project_folder, 'Block0202023', f'{year}_{archetype}') # create the destination path for copying the baseline scenario
        path_to_baseline = os.path.join(project_folder, 'Block0202023', year)
        try:
            shutil.copytree(path_to_baseline, path_to_new_scenario)
        except WindowsError:
            pass 
        archetype_path = os.path.join(project_folder, 'Block0202023', f'{year}_{archetype}', "inputs", "building-properties", "typology.dbf")
        archetype_file = gpd.read_file(archetype_path)
        archetype_file["STANDARD"] = archetype
        archetype_file.to_file(archetype_path)
        config.scenario_name = f'{year}_{archetype}'
        config.archetypes_mapper.buildings = list_buildings
        archetypes_mapper(locator=cea.inputlocator.InputLocator(config.scenario), 
                      update_architecture_dbf=True, update_air_conditioning_systems_dbf=True, 
                      update_indoor_comfort_dbf=True, update_internal_loads_dbf=True,
                      update_supply_systems_dbf=True, update_schedule_operation_cea=True, 
                      buildings=list_buildings)
        config.schedule_maker.buildings = list_buildings 
        schedule_maker(config) 
        radiation_main(config)
        config.demand.buildings = list_buildings
        demand_main(config) 

Running occupancy model for scenario C:\Users\felix\Programmieren\cea_berlin\CEA_model\Block0202023\2020_AB_E
Running occupancy model  with schedule model=deterministic
Using 11 CPU's
Schedule for building No. 1 completed out of 6: B1005
Schedule for building No. 2 completed out of 6: B1002
Schedule for building No. 3 completed out of 6: B1003
Schedule for building No. 4 completed out of 6: B1001
Schedule for building No. 5 completed out of 6: B1004
Schedule for building No. 6 completed out of 6: B1000
Using Daysim binaries from path: c:\users\felix\documents\cityenergyanalyst\dependencies\daysim\bin
Using Daysim data from path: c:\users\felix\documents\cityenergyanalyst\dependencies\daysim\lib
verifying geometry files
zone: C:\Users\felix\Programmieren\cea_berlin\CEA_model\Block0202023\2020_AB_E\inputs\building-geometry\zone.shp
surroundings: C:\Users\felix\Programmieren\cea_berlin\CEA_model\Block0202023\2020_AB_E\inputs\building-geometry\surroundings.shp
Getting geometry materials
Cr